In [1]:
import ee
try:
    ee.Initialize()
except:
    ee.Authenticate()
    ee.Initialize()
import geemap

Map = geemap.Map(center=[30, 111], zoom=10)

URLError: <urlopen error [WinError 10060] 由于连接方在一段时间后没有正确答复或连接的主机没有反应，连接尝试失败。>

In [ ]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import torch
import cv2

import gdal
import utils
import common
import dataset
import GetMapTiles
from models.ResNet import ResNet101

In [ ]:
# Adds Earth Engine dataset
worldBoundary = ee.FeatureCollection("users/liuph/shape/WorldBoundary")
ChinaBoundary = ee.FeatureCollection("users/410093033/China")

fuseImage = ee.ImageCollection('users/410093033/yizhixing').min().rename("fuseImage")
forest23 = fuseImage.expression("b(0)>=2&&b(0)<=3").rename("forest23")

# add landsat80------------------------------------------------------------------------------------
Landsat = ee.ImageCollection("LANDSAT/LC08/C01/T1_SR")
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','pixel_qa'])
LC = Landsat.filter(ee.Filter.calendarRange(2020, 2020, 'year')).select(selVar).map(utils.maskL8sr)
selVar = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])
LC = LC.select(selVar).median()

ndvi = LC.normalizedDifference(['B5', 'B4']).rename('NDVI')
ndwi = LC.normalizedDifference(['B3', 'B5']).rename('NDWI')
DEM = ee.Image("MERIT/DEM/v1_0_3")
terrain = ee.Algorithms.Terrain(DEM)
slope = terrain.select('slope')
stratified_classes = ndvi.expression('(b(0)/0.2)').int().rename('STRATIFIED_CLASSES')
LC_STN = LC.addBands(ndvi).addBands(ndwi).addBands(slope).addBands(stratified_classes).float()

selVar1 = ee.List(['B2', 'B3', 'B4', 'B5', 'B6', 'B7','NDVI','NDWI','slope'])


In [ ]:
path_fuseImage = r"./fused.tif"
path_ge = r"./ge.tif"

padding = 0.002

xmin = 111
ymin = 30
xmax = 111.1
ymax = 30.1

rect = ee.Algorithms.GeometryConstructors.Rectangle(ee.List([xmin, ymin, xmax, ymax]))

In [ ]:
if not os.path.exists(path_ge):
    GetMapTiles.getpic_tif(xmin-padding, ymax+padding, xmax+padding, ymin-padding, 18, source='google', out_filename=path_ge, style='s')
if not os.path.exists(path_fuseImage):
    geemap.ee_export_image(fuseImage, path_fuseImage, scale=30, region=rect)

In [6]:
llList_full = common.getLonLatListFromImage(path_fuseImage)
idx_unsure = np.where((llList_full[:, 2]==2) + (llList_full[:, 2]==3) + (llList_full[:, 2]==1))[0]
idx_sure = np.where((llList_full[:, 2]==0) + (llList_full[:, 2]==4) + (llList_full[:, 2]==5))[0]

llList_unsure = llList_full[idx_unsure]
llList_sure = llList_full[idx_sure]

In [7]:
dataLoader = dataset.getLonLatDataLoader(llList_unsure[:, 0:2], local=True, imgSavePath=path_ge, BSize=16, nWorkers=4, pinMem=True, APIKEY=None)
model = ResNet101(in_ch=3, n_classes=5)
pred = common.inference(model, data_loader=dataLoader, device=torch.device('cuda:0'), device_ids=(0,), comment="resnet101_330x330", save_path="./CKPT/")
predforest = (pred.cpu().numpy()==0).astype(np.int)

 15%|█▌        | 156/1027 [02:31<14:03,  1.03it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "C:\Users\thoma\AppData\Roaming\Python\Python37\site-packages\IPython\core\interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-169954e5748a>", line 3, in <module>
    pred = common.inference(model, data_loader=dataLoader, device=torch.device('cuda:0'), device_ids=(0,), comment="resnet101_330x330", save_path="./CKPT/")
  File "g:\repo\repo_py\GE_DensePredictionByGrid\common.py", line 29, in inference
    pred = torch.argmax(torch.softmax(model(img), dim=1), dim=1)
  File "C:\Program Files\Python37\lib\site-packages\torch\nn\modules\module.py", line 722, in _call_impl
    result = self.forward(*input, **kwargs)
  File "C:\Program Files\Python37\lib\site-packages\torch\nn\parallel\data_parallel.py", line 153, in forward
  

TypeError: object of type 'NoneType' has no len()

In [ ]:
llList_forest_sure = llList_sure.copy()
llList_forest_unsure = llList_unsure.copy()
llList_forest_sure[:, 2] = (llList_sure[:, 2]>=4).astype(np.int)
llList_forest_unsure[:, 2] = predforest

# select 5000 samples from sure and unsure area each
sampleNum = 5000
llList_forest_sure_sample = random.sample(llList_forest_sure.tolist(), sampleNum)
llList_forest_unsure_sample = random.sample(llList_forest_unsure.tolist(), sampleNum)
llList_forest_sample = llList_forest_sure_sample + llList_forest_unsure_sample

In [ ]:
CrdClsList = ee.List(llList_forest_sample)
mapped = CrdClsList.map(lambda x : ee.Feature(ee.Algorithms.GeometryConstructors.Point(ee.List(x).slice(0, 2)), {"forest":ee.List(x).get(2)}))
fc_points = ee.FeatureCollection(mapped)

sample_train = LC_STN.sampleRegions(collection=fc_points, geometries=True, scale=30)
classifier = ee.Classifier.smileRandomForest(numberOfTrees=200, variablesPerSplit=9, minLeafPopulation=1, bagFraction=0.5, maxNodes=None, seed=0).train(sample_train, "forest", selVar1)
predForest = LC_STN.clip(rect).select(selVar1).classify(classifier)

In [ ]:
Map = geemap.Map(center=[30, 111], zoom=10, add_google_map=True)

# Map.addLayer(ChinaBoundary, name="CB")
# Map.addLayer(forest23.updateMask(forest23.gt(0)), vis_params={"max":1, "min":0, "palette":['000000', '00FF00']}, name="forest23")
Map.addLayer(LC_STN.clip(rect), vis_params={"max":2000, "min":0, "bands":['B4', 'B3', 'B2']}, name="LC08")
Map.addLayer(predForest, vis_params={"max":1, "min":0, "palette":["FF0000", "00FF00"]}, name="pred", opacity=0.8)
Map.addLayer(fuseImage.clip(rect).updateMask(fuseImage.gt(0)), vis_params={"max":5, "min":0, "palette":["FFFFFF", "FF0000", "AA0000", "00FFaa", "00FF00", "00FF00"]}, opacity=0.8)

Map.add_basemap()
Map.addLayerControl()
Map